In [1]:
import gradio as gr
import tensorflow as tf
import numpy as np
from PIL import Image
import requests

ModuleNotFoundError: No module named 'gradio'

In [2]:
# check the version of gradio
print(gr.__version__)

2.5.2


In [3]:

# Download human-readable labels for ImageNet.
response = requests.get("https://git.io/JJkYN")
labels = response.text.split("\n")

In [ ]:
print(len(labels))

1001


In [ ]:
# Download the weights of the model trained on ImageNet.

mobile_net = tf.keras.applications.MobileNetV2()
xception_net = tf.keras.applications.Xception()

In [ ]:
def classify_image_with_mobile_net(im):
    im = Image.fromarray(im.astype('uint8'), 'RGB')
    im = im.resize((224, 224))
    arr = np.array(im).reshape((-1, 224, 224, 3))
    arr = tf.keras.applications.mobilenet.preprocess_input(arr)
    prediction = mobile_net.predict(arr).flatten()
    return {labels[i]: float(prediction[i]) for i in range(1000)}
    

In [ ]:
def classify_image_with_xception_net(im):
    # Resize the image to
    im = Image.fromarray(im.astype('uint8'), 'RGB')
    im = im.resize((299, 299))
    arr = np.array(im).reshape((-1, 299, 299, 3))
    arr = tf.keras.applications.xception.preprocess_input(arr)
    prediction = xception_net.predict(arr).flatten()
    return {labels[i]: float(prediction[i]) for i in range(1000)}

In [ ]:
imagein = gr.inputs.Image()
label = gr.outputs.Label(num_top_classes=3)
sample_images = [
                 ["images/monkey.jpg"],
                 ["images/cat.jpg"],
                 ["images/eagle.jpg"],
                 ["images/monkey.jpg"],
                 ["images/dog.jpg"],
                 ["images/palace.jpg"],
                 ["images/bird.jpg"],
                 ["images/plane.jpg"],
                 ["images/masa.jpg"],
                 ["images/bird_breed.jpg"]
]

In [ ]:
# # Buiding the UI

gr.Interface(
    [classify_image_with_mobile_net, classify_image_with_xception_net],
    imagein,
    label,
    title="MobileNetV2 vs. InceptionNet",
    description="""We compare two (2) different state-of-the-art deep learning models that classify ImageNet dataset into one of 1000 categories: MobileNetV2
    vs. Xception in order to demonstrate the trade-off between model performance and inference""",
examples=sample_images, interpretation='default', theme='darkhuggingface').launch(share=True)